Imports

In [83]:
import ir_datasets
import logging
import sys
import json
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import RSLPStemmer
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jopec\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Configurando LOGGER

In [84]:
logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                    format='[%(asctime)s]%(levelname)s(%(name)s): %(message)s')
LOGGER = logging.getLogger(__name__)

Importando dataset

In [85]:
dataset = ir_datasets.load("highwire/trec-genomics-2006")

Processando map_consultas

In [86]:
map_consultas = {}
map_query_id_to_index = {}

In [87]:
i = 0
for query in dataset.queries_iter():
    # Mapeia o ID original da consulta (string) para o nosso índice numérico (i)
    # Ex: map_query_id_to_index['q1'] = 0
    map_query_id_to_index[query.query_id] = i

    # Processa o texto da consulta
    text = query.text
    processed_text = word_tokenize(text.lower())
    processed_text = ' '.join([word for word in processed_text if word.isalpha() and word not in stop_words])
    
    # Armazena a consulta processada usando o índice numérico como chave
    # Ex: map_consultas[0] = "texto processado da consulta q1"
    map_consultas[i] = processed_text

    LOGGER.info(f"Processada consulta ID '{query.query_id}' para índice {i}: {processed_text}")
    
    i += 1

[2025-07-22 07:36:35,160]INFO(__main__): Processada consulta ID '160' para índice 0: role prnp mad cow disease
[2025-07-22 07:36:35,161]INFO(__main__): Processada consulta ID '161' para índice 1: role ide alzheimer disease
[2025-07-22 07:36:35,161]INFO(__main__): Processada consulta ID '162' para índice 2: role cancer
[2025-07-22 07:36:35,162]INFO(__main__): Processada consulta ID '163' para índice 3: role apc adenomatous polyposis coli colon cancer
[2025-07-22 07:36:35,162]INFO(__main__): Processada consulta ID '164' para índice 4: role parkinson disease
[2025-07-22 07:36:35,163]INFO(__main__): Processada consulta ID '165' para índice 5: cathepsin ctsd apolipoprotein e apoe interactions contribute alzheimer disease
[2025-07-22 07:36:35,163]INFO(__main__): Processada consulta ID '166' para índice 6: role transforming growth cerebral amyloid angiopathy caa
[2025-07-22 07:36:35,164]INFO(__main__): Processada consulta ID '167' para índice 7: nucleoside diphosphate kinase contribute tumor 

Salvando map_consultas como .json

In [88]:
with open('map_consultas.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_consultas,      # Dicionário de consultas processadas
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhorar legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

Processando map_documentos

In [89]:
map_documentos = {}
map_index = {}

In [ ]:
i = 0
for doc in dataset.docs_iter():
    fields_to_join = [
        doc.title,
        doc.journal
    ]
    
    # Junta todos os campos selecionados em uma única string, ignorando campos vazios (None)
    text = ' '.join(str(field) for field in fields_to_join if field)
    
    # Converte para minúsculas e tokeniza o texto
    processed_text_tokens = word_tokenize(text.lower())
    
    # Remove stopwords e palavras que não são puramente alfabéticas
    cleaned_tokens = [word for word in processed_text_tokens if word.isalpha() and word not in stop_words]
    
    # Junta os tokens limpos de volta em uma única string
    processed_text = ' '.join(cleaned_tokens)
    
    # Adiciona o documento processado ao dicionário, usando o novo índice 'i' como chave
    map_documentos[i] = processed_text
    
    # Esta linha continua útil para mapear o ID antigo para o novo
    map_index[doc.doc_id] = i
    
    i += 1
    

Salvando map_documentos como .json

In [91]:
with open('map_documentos.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_documentos,     # Dicionário de documentos processados
        out_file_opened,    # Objeto do arquivo
        indent=2,           # Indentação para melhorar legibilidade
        ensure_ascii=False  # Permite caracteres não-ASCII
    )

Processando map_consultas_relevancias

In [92]:
map_consultas_relevancias = {}

In [93]:
for qrel in dataset.qrels_iter():
    # 1. Procura o ID original do documento (qrel.doc_id) no seu mapa de documentos.
    #    Isso retorna o índice numérico do documento (ex: 42).
    doc_index = map_index.get(qrel.doc_id)

    # 2. Procura o ID original da consulta (qrel.query_id) no nosso novo mapa.
    #    Isso retorna o índice numérico da consulta (ex: 0).
    query_index = map_query_id_to_index.get(qrel.query_id)

    # 3. Executa o bloco somente se AMBOS, o documento e a consulta, existirem em nossos mapas.
    if doc_index is not None and query_index is not None:
        
        # 4. Garante que a consulta (pelo seu índice numérico) tenha uma entrada no dicionário.
        #    Se for a primeira vez que vemos a consulta de índice '0', cria um dicionário vazio para ela.
        if query_index not in map_consultas_relevancias:
            map_consultas_relevancias[query_index] = {}

        # 5. Armazena a pontuação de relevância usando os índices numéricos.
        #    A estrutura final será: map_consultas_relevancias[0][42] = 1.0
        map_consultas_relevancias[query_index][doc_index] = qrel.relevance


Salvando map_consultas_relevancias como .json

In [94]:
with open('map_consultas_relevancias.json', 'w', encoding='utf-8') as out_file_opened:
    json.dump(
        map_consultas_relevancias, # Dicionário de relevâncias processadas
        out_file_opened,           # Objeto do arquivo
        indent=2,                  # Indentação para melhorar legibilidade
        ensure_ascii=False         # Permite caracteres não-ASCII
    )